In [ ]:
# To install selenium:
!pip install selenium

In [27]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Chrome()
driver.get('https://www.linkedin.com/jobs/search/?currentJobId=3871191552&distance=25&f_T=340%2C25190%2C25189%2C2732%2C1547&geoId=102007122&keywords=data%20scientist&origin=JOB_SEARCH_PAGE_JOB_FILTER&refresh=true&sortBy=R')
time.sleep(2)

job_title = []
company_name = []
data_time = []
job_link = []
seniority_level = []  #job_type
employment_type = []

while True:
    # Fetch job elements
    title = driver.find_elements(By.CLASS_NAME, "base-search-card__title")
    company = driver.find_elements(By.CLASS_NAME, "base-search-card__subtitle")
    data_elements = driver.find_elements(By.CLASS_NAME, "job-search-card__listdate")
    link = driver.find_elements(By.CSS_SELECTOR, "a.base-card__full-link")

    # Ensure that the loop runs only up to the minimum length of the lists
    for i in range(min(len(title), len(company), len(link), len(data_elements))):
        job_title.append(title[i].text)
        company_name.append(company[i].text)
        job_link.append(link[i].get_attribute('href'))

        # Extract the value of the 'datetime' attribute and store it in data_time
        date_time_attr = data_elements[i].get_attribute('datetime')
        data_time.append(date_time_attr if date_time_attr else "No Date Available")

    # Scroll and try to click the "See More Jobs" button
    try:
        see_more_button = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, "//*[contains(text(), 'See more jobs')]"))
        )

        # Scroll to the button and click it
        driver.execute_script("arguments[0].scrollIntoView();", see_more_button)
        time.sleep(1)
        see_more_button.click()
        time.sleep(3)  # Wait for new jobs to load

    except Exception as e:
        # If the button is not found, keep scrolling until the end
        driver.execute_script("window.scrollBy(0, 1500);")
        time.sleep(2)

    # Check if the end of the page has been reached
    current_height = driver.execute_script("return window.pageYOffset + window.innerHeight;")
    total_height = driver.execute_script("return document.body.scrollHeight;")

    if current_height >= total_height:
        break  # Exit the loop if no more content is left to scroll through

# Visit each job link and extract additional data
for l in job_link:
    driver.get(l)
    seniority = driver.find_elements(By.CSS_SELECTOR, "ul.description__job-criteria-list > li.description__job-criteria-item:first-child > span")
    employment = driver.find_elements(By.CSS_SELECTOR, "ul.description__job-criteria-list > li.description__job-criteria-item:nth-child(2) > span")
    # Ensure that there are elements before trying to access them
    if seniority:
        seniority_level.append(seniority[0].text)  # Assuming you want the first element
        employment_type.append(employment[0].text)
    else:
        seniority_level.append("No job type available")
        employment_type.append("No job type available")


    time.sleep(2)  # Wait for the page to load

    driver.back()
    time.sleep(2)  # Wait for the page to load before continuing

driver.quit()


In [40]:
import csv
from itertools import zip_longest

with open('DS_Companies.csv', 'w', encoding='utf-8') as csvfile:
    DS_Companies = csv.writer(csvfile)
    
    DS_Companies.writerow(['Job_Title', 'Company_Name', 'data_time', 'seniority_level', 'employment_type', 'job_link'])
    DS_Companies.writerows(zip_longest(job_title, company_name, data_time, seniority_level, employment_type, job_link))
